In [1]:
import pandas as pd 
import numpy as np
import json, urllib.request
from datetime import datetime
from datetime import timedelta
from collections import Counter

transaction_data = pd.read_json('transaction-data-adhoc-analysis.json')


In [2]:
months_loop = [pd.to_datetime(x).strftime("%m") for x in list(transaction_data['transaction_date'])]
transaction_data.insert(2,"Months",months_loop, True)


In [3]:
table_1_2 = transaction_data.drop(["transaction_date","address","birthdate","mail","name","username","sex"], axis = 1, inplace=False)
table_1_2.head()

,Months,transaction_items,transaction_value
0,01,"Exotic Extras,Beef Chicharon,(x4)",5196
1,01,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056
2,01,"HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3...",5500
3,01,"HealthyKid 3+,Yummy Vegetables,(x1)",500
4,01,"Candy City,Orange Beans,(x3);Candy City,Gummy ...",2697


In [4]:
new_column = table_1_2["transaction_items"].str.split(";", expand = True)
new_column

,0,1,2
0,"Exotic Extras,Beef Chicharon,(x4)",None,None
1,"Exotic Extras,Beef Chicharon,(x3)","HealthyKid 3+,Nutrional Milk,(x4)","Candy City,Orange Beans,(x1)"
2,"HealthyKid 3+,Gummy Vitamins,(x3)","HealthyKid 3+,Yummy Vegetables,(x2)",None
3,"HealthyKid 3+,Yummy Vegetables,(x1)",None,None
4,"Candy City,Orange Beans,(x3)","Candy City,Gummy Worms,(x4)","HealthyKid 3+,Gummy Vitamins,(x1)"
...,...,...,...
83031,"Exotic Extras,Kimchi and Seaweed,(x2)","HealthyKid 3+,Nutrional Milk,(x1)",None
83032,"HealthyKid 3+,Yummy Vegetables,(x1)","Exotic Extras,Kimchi and Seaweed,(x1)","Candy City,Orange Beans,(x4)"
83033,"Candy City,Orange Beans,(x4)","Candy City,Gummy Worms,(x4)","HealthyKid 3+,Yummy Vegetables,(x4)"
83034,"HealthyKid 3+,Nutrional Milk,(x2)","Candy City,Gummy Worms,(x2)",None


In [5]:
table_1_2["First Product"]= new_column[0]

table_1_2["Second Product"]= new_column[1]

table_1_2["Third Product"]= new_column[2]
  
table_1_2.drop(columns =["transaction_items"], inplace = True)

table_1_2.head()

,Months,transaction_value,First Product,Second Product,Third Product
0,01,5196,"Exotic Extras,Beef Chicharon,(x4)",None,None
1,01,12056,"Exotic Extras,Beef Chicharon,(x3)","HealthyKid 3+,Nutrional Milk,(x4)","Candy City,Orange Beans,(x1)"
2,01,5500,"HealthyKid 3+,Gummy Vitamins,(x3)","HealthyKid 3+,Yummy Vegetables,(x2)",None
3,01,500,"HealthyKid 3+,Yummy Vegetables,(x1)",None,None
4,01,2697,"Candy City,Orange Beans,(x3)","Candy City,Gummy Worms,(x4)","HealthyKid 3+,Gummy Vitamins,(x1)"


In [6]:
first_product = table_1_2["First Product"].str.split(",", expand = True)
second_product = table_1_2["Second Product"].str.split(",", expand = True)
third_product = table_1_2["Third Product"].str.split(",", expand = True)

#First Product
table_1_2["First Product Brand"]= first_product[0]

table_1_2["First Product Item"]= first_product[1]

table_1_2["First Product Count"]= first_product[2]

#Second Product
table_1_2["Second Product Brand"]= second_product[0]

table_1_2["Second Product Item"]= second_product[1]

table_1_2["Second Product Count"]= second_product[2]

#Third Product
table_1_2["Third Product Brand"]= third_product[0]

table_1_2["Third Product Item"]= third_product[1]

table_1_2["Third Product Count"]= third_product[2]

#drop the original columns
table_1_2.drop(columns =["First Product"], inplace = True)
table_1_2.drop(columns =["Second Product"], inplace = True)
table_1_2.drop(columns =["Third Product"], inplace = True)

table_1_2.head()

,Months,transaction_value,First Product Brand,First Product Item,First Product Count,Second Product Brand,Second Product Item,Second Product Count,Third Product Brand,Third Product Item,Third Product Count
0,01,5196,Exotic Extras,Beef Chicharon,(x4),None,None,None,None,None,None
1,01,12056,Exotic Extras,Beef Chicharon,(x3),HealthyKid 3+,Nutrional Milk,(x4),Candy City,Orange Beans,(x1)
2,01,5500,HealthyKid 3+,Gummy Vitamins,(x3),HealthyKid 3+,Yummy Vegetables,(x2),None,None,None
3,01,500,HealthyKid 3+,Yummy Vegetables,(x1),None,None,None,None,None,None
4,01,2697,Candy City,Orange Beans,(x3),Candy City,Gummy Worms,(x4),HealthyKid 3+,Gummy Vitamins,(x1)


In [7]:
table_1_2.fillna(value=np.nan, inplace=True)
table_1_2["First Product Count"]=table_1_2["First Product Count"].apply(lambda x: str(x).replace('(','').replace('x','').replace(')',''))
table_1_2["Second Product Count"] = [str(x).replace('(','').replace('x','').replace(')','') for x in table_1_2["Second Product Count"]]
table_1_2["Third Product Count"] = [str(x).replace('(','').replace('x','').replace(')','') for x in table_1_2["Third Product Count"]]
table_1_2 = table_1_2.apply(lambda x: x.replace('nan','0'))

In [8]:
table_1_2["First Product Count"] = [int(x) for x in table_1_2["First Product Count"]]
table_1_2["Second Product Count"] = [int(x) for x in table_1_2["Second Product Count"]]
table_1_2["Third Product Count"] = [int(x) for x in table_1_2["Third Product Count"]]
table_1_2.head()

,Months,transaction_value,First Product Brand,First Product Item,First Product Count,Second Product Brand,Second Product Item,Second Product Count,Third Product Brand,Third Product Item,Third Product Count
0,01,5196,Exotic Extras,Beef Chicharon,4,NaN,NaN,0,NaN,NaN,0
1,01,12056,Exotic Extras,Beef Chicharon,3,HealthyKid 3+,Nutrional Milk,4,Candy City,Orange Beans,1
2,01,5500,HealthyKid 3+,Gummy Vitamins,3,HealthyKid 3+,Yummy Vegetables,2,NaN,NaN,0
3,01,500,HealthyKid 3+,Yummy Vegetables,1,NaN,NaN,0,NaN,NaN,0
4,01,2697,Candy City,Orange Beans,3,Candy City,Gummy Worms,4,HealthyKid 3+,Gummy Vitamins,1


In [9]:
table_1_f = table_1_2
table_1_f["First Product Brand"] = table_1_f[["First Product Brand", "First Product Item"]].apply(" | ".join, axis=1)
table_1_f.drop(["First Product Item"], axis = 1, inplace=True)
table_1_f["Second Product Brand"] = table_1_f["Second Product Brand"].map(str) + " | " + table_1_f["Second Product Item"].map(str)
table_1_f.drop(["Second Product Item"], axis = 1, inplace=True)
table_1_f["Third Product Brand"] = table_1_f["Third Product Brand"].map(str) + " | " +table_1_f["Third Product Item"].map(str)
table_1_f.drop(["Third Product Item"], axis = 1, inplace=True)
table_1_f.head()


,Months,transaction_value,First Product Brand,First Product Count,Second Product Brand,Second Product Count,Third Product Brand,Third Product Count
0,01,5196,Exotic Extras | Beef Chicharon,4,nan | nan,0,nan | nan,0
1,01,12056,Exotic Extras | Beef Chicharon,3,HealthyKid 3+ | Nutrional Milk,4,Candy City | Orange Beans,1
2,01,5500,HealthyKid 3+ | Gummy Vitamins,3,HealthyKid 3+ | Yummy Vegetables,2,nan | nan,0
3,01,500,HealthyKid 3+ | Yummy Vegetables,1,nan | nan,0,nan | nan,0
4,01,2697,Candy City | Orange Beans,3,Candy City | Gummy Worms,4,HealthyKid 3+ | Gummy Vitamins,1


In [10]:
table_1_f = table_1_f.apply(lambda x: x.replace('nan | nan','No Item'))
table_1_f.head(7)

,Months,transaction_value,First Product Brand,First Product Count,Second Product Brand,Second Product Count,Third Product Brand,Third Product Count
0,01,5196,Exotic Extras | Beef Chicharon,4,No Item,0,No Item,0
1,01,12056,Exotic Extras | Beef Chicharon,3,HealthyKid 3+ | Nutrional Milk,4,Candy City | Orange Beans,1
2,01,5500,HealthyKid 3+ | Gummy Vitamins,3,HealthyKid 3+ | Yummy Vegetables,2,No Item,0
3,01,500,HealthyKid 3+ | Yummy Vegetables,1,No Item,0,No Item,0
4,01,2697,Candy City | Orange Beans,3,Candy City | Gummy Worms,4,HealthyKid 3+ | Gummy Vitamins,1
5,01,2990,HealthyKid 3+ | Nutrional Milk,1,HealthyKid 3+ | Yummy Vegetables,2,No Item,0
6,01,3196,Exotic Extras | Kimchi and Seaweed,4,No Item,0,No Item,0


In [11]:
# 1 - Exotic Extras | Beef Chicharon
# 2 - HealthyKid 3+ | Nutrional Milk
# 3 - Candy City | Orange Beans
# 4 - HealthyKid 3+ | Gummy Vitamins
# 5 - HealthyKid 3+ | Yummy Vegetables
# 6 - Candy City | Gummy Worms
# 7 - Exotic Extras | Kimchi and Seaweed	

count_array = np.array([[4,0,0,0,0,0,0],
                        [3,4,1,0,0,0,0],
                        [0,0,0,3,2,0,0],
                        [0,0,0,0,1,0,0],
                        [0,0,3,1,0,4,0],
                        [0,1,0,0,2,0,0],
                        [0,0,0,0,0,0,4]])

price_array = np.array([5196,12056,5500,500,2697,2990,3196])

soln_array = np.linalg.solve(count_array,price_array)

In [12]:
def item_price(x):
    if x == "Exotic Extras | Beef Chicharon":
        return int(1299)
    elif x == "HealthyKid 3+ | Nutrional Milk":
        return int(1990)
    elif x == "Candy City | Orange Beans":
        return int(199)
    elif x == "HealthyKid 3+ | Gummy Vitamins":
        return int(1500)
    elif x == "HealthyKid 3+ | Yummy Vegetables":
        return int(500)
    elif x == "Candy City | Gummy Worms":
        return int(150)
    elif x == "Exotic Extras | Kimchi and Seaweed":
        return int(799)
    else:
        return int(0)

In [13]:
table_1_final_ver = table_1_f[['Months','First Product Brand','First Product Count']].copy()
table_1_final_ver["Product Price"] = table_1_final_ver['First Product Brand'].apply(item_price)
table_1_final_ver["Total Product Price"] = table_1_final_ver['First Product Count']*table_1_final_ver["Product Price"]
table_1_final_ver.drop(["Product Price"], axis = 1, inplace=True)
table_1_final_ver

,Months,First Product Brand,First Product Count,Total Product Price
0,01,Exotic Extras | Beef Chicharon,4,5196
1,01,Exotic Extras | Beef Chicharon,3,3897
2,01,HealthyKid 3+ | Gummy Vitamins,3,4500
3,01,HealthyKid 3+ | Yummy Vegetables,1,500
4,01,Candy City | Orange Beans,3,597
...,...,...,...,...
83031,06,Exotic Extras | Kimchi and Seaweed,2,1598
83032,06,HealthyKid 3+ | Yummy Vegetables,1,500
83033,06,Candy City | Orange Beans,4,796
83034,06,HealthyKid 3+ | Nutrional Milk,2,3980


In [14]:
table_2_final_ver = table_1_f[['Months','Second Product Brand','Second Product Count']].copy()
table_2_final_ver["Product Price"] = table_2_final_ver['Second Product Brand'].apply(item_price)
table_2_final_ver["Total Product Price"] = table_2_final_ver['Second Product Count']*table_2_final_ver["Product Price"]
table_2_final_ver.drop(["Product Price"], axis = 1, inplace=True)
table_2_final_ver

,Months,Second Product Brand,Second Product Count,Total Product Price
0,01,No Item,0,0
1,01,HealthyKid 3+ | Nutrional Milk,4,7960
2,01,HealthyKid 3+ | Yummy Vegetables,2,1000
3,01,No Item,0,0
4,01,Candy City | Gummy Worms,4,600
...,...,...,...,...
83031,06,HealthyKid 3+ | Nutrional Milk,1,1990
83032,06,Exotic Extras | Kimchi and Seaweed,1,799
83033,06,Candy City | Gummy Worms,4,600
83034,06,Candy City | Gummy Worms,2,300


In [15]:
table_3_final_ver = table_1_f[['Months','Third Product Brand','Third Product Count']].copy()
table_3_final_ver["Product Price"] = table_3_final_ver['Third Product Brand'].apply(item_price)
table_3_final_ver["Total Product Price"] = table_3_final_ver['Third Product Count']*table_3_final_ver["Product Price"]
table_3_final_ver.drop(["Product Price"], axis = 1, inplace=True)
table_3_final_ver

,Months,Third Product Brand,Third Product Count,Total Product Price
0,01,No Item,0,0
1,01,Candy City | Orange Beans,1,199
2,01,No Item,0,0
3,01,No Item,0,0
4,01,HealthyKid 3+ | Gummy Vitamins,1,1500
...,...,...,...,...
83031,06,No Item,0,0
83032,06,Candy City | Orange Beans,4,796
83033,06,HealthyKid 3+ | Yummy Vegetables,4,2000
83034,06,No Item,0,0


In [16]:
answer_1 = pd.pivot_table(table_1_final_ver, index = ['First Product Brand'],
                          columns = 'Months',
                          values = 'Total Product Price',
                          aggfunc = ['sum'],
                          fill_value=None,
                          margins = False,
                          dropna = True,
                          margins_name = 'All',
                          observed = False,
                          sort = True)
answer_1

sum                             \
Months                                   01       02       03       04   
First Product Brand                                                      
Candy City | Gummy Worms             726600   722700   776700   732150   
Candy City | Orange Beans            984453  1027238   951817   991418   
Exotic Extras | Beef Chicharon      6439143  6623601  6252087  6362502   
Exotic Extras | Kimchi and Seaweed  3880743  3899919  4032553  3892728   
HealthyKid 3+ | Gummy Vitamins      7279500  7617000  7630500  7288500   
HealthyKid 3+ | Nutrional Milk      9661450  9639560  9466430  9522150   
HealthyKid 3+ | Yummy Vegetables    2540000  2465000  2444000  2520500   

                                                      
Months                                   05       06  
First Product Brand                                   
Candy City | Gummy Worms             741900   753600  
Candy City | Orange Beans            994801   999776  
Exotic Extras | Beef Chicharon      6423555  6292356  
Exotic Extras | Kimchi and Seaweed  3882341  4032553  
HealthyKid 3+ | Gummy Vitamins      7467000  7348500  
HealthyKid 3+ | Nutrional Milk      9663440  9518170  
HealthyKid 3+ | Yummy Vegetables    2482500  2485500

In [17]:
answer_2 = pd.pivot_table(table_2_final_ver, index = ['Second Product Brand'],
                          columns = 'Months',
                          values = 'Total Product Price',
                          aggfunc = ['sum'],
                          fill_value=None,
                          margins = False,
                          dropna = True,
                          margins_name = 'All',
                          observed = False,
                          sort = True)
answer_2

sum                             \
Months                                   01       02       03       04   
Second Product Brand                                                     
Candy City | Gummy Worms             480150   504150   476100   499050   
Candy City | Orange Beans            665257   653715   643168   668043   
Exotic Extras | Beef Chicharon      4082757  4316577  4399713  4269813   
Exotic Extras | Kimchi and Seaweed  2554403  2670258  2631906  2753354   
HealthyKid 3+ | Gummy Vitamins      4674000  4944000  5005500  4920000   
HealthyKid 3+ | Nutrional Milk      6568990  6373970  6897340  6547100   
HealthyKid 3+ | Yummy Vegetables    1639000  1831000  1649500  1613500   
No Item                                   0        0        0        0   

                                                      
Months                                   05       06  
Second Product Brand                                  
Candy City | Gummy Worms             484200   492150  
Candy City | Orange Beans            666849   657297  
Exotic Extras | Beef Chicharon      4393218  4355547  
Exotic Extras | Kimchi and Seaweed  2574378  2715002  
HealthyKid 3+ | Gummy Vitamins      4806000  4975500  
HealthyKid 3+ | Nutrional Milk      6732170  6636650  
HealthyKid 3+ | Yummy Vegetables    1577000  1610000  
No Item                                   0        0

In [18]:
answer_3 = pd.pivot_table(table_3_final_ver, index = ['Third Product Brand'],
                          columns = 'Months',
                          values = 'Total Product Price',
                          aggfunc = ['sum'],
                          fill_value=None,
                          margins = False,
                          dropna = True,
                          margins_name = 'All',
                          observed = False,
                          sort = True)
answer_3

sum                             \
Months                                   01       02       03       04   
Third Product Brand                                                      
Candy City | Gummy Worms             227100   272550   245100   275250   
Candy City | Orange Beans            295316   316410   317604   313425   
Exotic Extras | Beef Chicharon      2032935  2051121  2099184  2214795   
Exotic Extras | Kimchi and Seaweed  1295978  1379074  1299174  1280797   
HealthyKid 3+ | Gummy Vitamins      2568000  2409000  2581500  2554500   
HealthyKid 3+ | Nutrional Milk      3126290  3271560  3289470  3404890   
HealthyKid 3+ | Yummy Vegetables     800500   832000   854500   796500   
No Item                                   0        0        0        0   

                                                      
Months                                   05       06  
Third Product Brand                                   
Candy City | Gummy Worms             244050   244350  
Candy City | Orange Beans            321186   354021  
Exotic Extras | Beef Chicharon      2209599  2214795  
Exotic Extras | Kimchi and Seaweed  1351908  1325541  
HealthyKid 3+ | Gummy Vitamins      2649000  2646000  
HealthyKid 3+ | Nutrional Milk      3267580  3281510  
HealthyKid 3+ | Yummy Vegetables     808000   765500  
No Item                                   0        0

In [19]:
#table 2: breakdown of the total sale value per item per month
final_answer = answer_1 + answer_2 + answer_3
final_answer = final_answer.drop('No Item', axis = 0)
final_answer

sum                          \
Months                                      01          02          03   
Candy City | Gummy Worms             1433850.0   1499400.0   1497900.0   
Candy City | Orange Beans            1945026.0   1997363.0   1912589.0   
Exotic Extras | Beef Chicharon      12554835.0  12991299.0  12750984.0   
Exotic Extras | Kimchi and Seaweed   7731124.0   7949251.0   7963633.0   
HealthyKid 3+ | Gummy Vitamins      14521500.0  14970000.0  15217500.0   
HealthyKid 3+ | Nutrional Milk      19356730.0  19285090.0  19653240.0   
HealthyKid 3+ | Yummy Vegetables     4979500.0   5128000.0   4948000.0   

                                                                        
Months                                      04          05          06  
Candy City | Gummy Worms             1506450.0   1470150.0   1490100.0  
Candy City | Orange Beans            1972886.0   1982836.0   2011094.0  
Exotic Extras | Beef Chicharon      12847110.0  13026372.0  12862698.0  
Exotic Extras | Kimchi and Seaweed   7926879.0   7808627.0   8073096.0  
HealthyKid 3+ | Gummy Vitamins      14763000.0  14922000.0  14970000.0  
HealthyKid 3+ | Nutrional Milk      19474140.0  19663190.0  19436330.0  
HealthyKid 3+ | Yummy Vegetables     4930500.0   4867500.0   4861000.0